# Mustererkennung/Machine Learning - Assignment 6



In [77]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split

### Load the spam dataset:

In [78]:
data = np.array(pd.read_csv('spambase.data', header=None))

X = data[:,:-1] # features
y = data[:,-1] # Last column is label

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, shuffle=True, stratify=y)

In [79]:
def accuracy(y_true, y_pred):
    return np.mean(y_true == y_pred)

def gini(y_true):
    """
    For simplicity reasons this assumes that there are only 2 classes
    """
    y_true = np.array(y_true)
    p_m0 = np.mean((y_true == 0) * 1)
    p_m1 = np.mean((y_true == 1) * 1)
    return 1 - p_m0 ** 2 - p_m1 ** 2

class LeafNode():
    def fit(self, c):
        self.c = c
        
    def predict(self, x):
        return self.c
    
class InternalNode():
    def fit(self, x, y, depth, max_depth, max_features_to_sample):
        m, n = x.shape
        # columns are j, split_index, loss_total
        split_infos = []
        if max_features_to_sample == None:
            iter_over = range(n)
        else:
            # in RandomForest we build the tree only on a subset of the features at each Node
            iter_over = np.random.permutation(n)[:max_features_to_sample]
            
        for j in iter_over:
            # sort rows by feature j in ascending order
            x = x[x[:,j].argsort()]
            last_val = None
            for split_index in range(0, m - 1):
                this_val = x[split_index, j]
                y_top_split = y[:split_index + 1]
                y_bottom_split = y[split_index + 1:]
                
                loss_1 = gini(y_top_split)
                loss_2 = gini(y_bottom_split)
                
                # use weighted average
                loss_total = (y_top_split.shape[0] / m) * loss_1 + (y_bottom_split.shape[0] / m) * loss_2
                
                # if it's the same split, we will just overwrite the index to split at
                if last_val != None and last_val == this_val:
                    split_infos[-1][1] = split_index
                else:
                    row = np.array([j, split_index, loss_total])
                    split_infos.append(row)
                last_val = this_val
                
        split_infos = np.array(split_infos)
        best_split_idx = np.argmin(split_infos[:,-1], axis=0)
        best_split = split_infos[best_split_idx]
        self.j = int(best_split[0])
        split_index = int(best_split[1])
        x = x[x[:, self.j].argsort()]
        x_top_split, y_top_split = x[:split_index + 1], y[:split_index + 1]
        x_bottom_split, y_bottom_split = x[split_index + 1:], y[split_index + 1:]
        
        self.z = x_top_split[-1, self.j] + (x_bottom_split[0, self.j] - x_top_split[-1, self.j]) / 2
        
        if x_top_split.shape[0] <= 1 or depth >= max_depth:
            self.left_child = LeafNode()
            c = self.find_c(y_top_split)
            self.left_child.fit(c)
        else:
            self.left_child = InternalNode()
            self.left_child.fit(x_top_split, y_top_split, depth + 1, max_depth, max_features_to_sample)
            
        if x_bottom_split.shape[0] <= 1 or depth >= max_depth:
            self.right_child = LeafNode()
            c = self.find_c(y_bottom_split)
            self.right_child.fit(c)
        else:
            self.right_child = InternalNode()
            self.right_child.fit(x_bottom_split, y_bottom_split, depth + 1, max_depth, max_features_to_sample)
        
    def predict(self, x):
        if x[self.j] <= self.z:
            return self.left_child.predict(x)
        return self.right_child.predict(x)
    
    def find_c(self, y):
        """
        For simplicity reasons this assumes that there are only 2 classes
        """
        zeros = np.sum(y == 0)
        ones = np.sum(y == 1)
        if zeros > ones:
            return 1
        return 0
        
    
class DecisionTreeClassifier():
    """
    Basically just holds the root node of the tree which starts the recursion
    features_to_sample will be set when creating a RandomForest 
    """
    def __init__(self, max_depth, features_to_sample=None):
        self.max_depth = max_depth
        self.features_to_sample = features_to_sample
        
    def fit(self, x, y):
        self.root = InternalNode()
        x = np.copy(x)
        y = np.copy(y)
        self.root.fit(x, y, 1, self.max_depth, self.features_to_sample)
    
    def predict(self, x):
        y_preds = []
        for sample in x:
            y_pred = self.root.predict(sample)
            y_preds.append(y_pred)
        return np.array(y_preds)


This takes about two minutes

1min 43s and test acc of 0.639 for max_depth = 5 

In [67]:
%%time

clf = DecisionTreeClassifier(max_depth = 100)
clf.fit(X_train[:300], y_train[:300])
print()


CPU times: user 15.7 s, sys: 12 ms, total: 15.8 s
Wall time: 15.8 s


In [68]:
y_pred = clf.predict(X_test)
acc = accuracy(y_test, y_pred)
print(acc)
print(np.sum(y_pred == 0))

0.5699391833188532
586


In [80]:
# TODO Delete later
from tqdm.notebook import tqdm

class RandomForestClassifier():
    def __init__(self, num_trees, bootstrap_ratio, max_depth):
        self.num_trees = num_trees
        self.bootstrap_ratio = bootstrap_ratio
        self.max_depth = max_depth
    
    def boostrap_dataset(self, x, y):
        """
        Creates a dataset with m * bootstrap_ratio samples
        drawn with replacement
        """
        size = int(x.shape[0] * self.bootstrap_ratio)
        row_indices = np.random.randint(low=0, high=x.shape[0], size=(size, ))
        return x[row_indices], y[row_indices]
            
    def fit(self, x, y):
        x = np.copy(x)
        y = np.copy(y)
        self.trees = []
        num_features = int(np.sqrt(x.shape[1]))
        for tree in tqdm(range(self.num_trees)):
            x_sub, y_sub = self.boostrap_dataset(x, y)
            weak_predictor = DecisionTreeClassifier(self.max_depth, num_features)
            weak_predictor.fit(x_sub, y_sub)
            self.trees.append(weak_predictor)
        
    def predict(self, x):
        y_preds = []
        for sample in x:
            votes = []
            for tree in self.trees:
                # decision tree expects matrix as input
                sample = sample.reshape((1, -1))
                votes.append(tree.predict(sample))
            votes = np.array(votes)
            if np.mean(votes) > 0.5:
                y_preds.append(1)
            else:
                y_preds.append(0)
        return np.array(y_preds)

In [87]:
%%time
clf = RandomForestClassifier(num_trees=20, bootstrap_ratio=1, max_depth=60)
clf.fit(X_train[:300], y_train[:300])
print()



CPU times: user 34.5 s, sys: 59.9 ms, total: 34.6 s
Wall time: 34.5 s


In [88]:
y_pred = clf.predict(X_test)
acc = accuracy(y_test, y_pred)
print(acc)
print(np.sum(y_pred == 0))

0.4456993918331885
315


In [89]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=20, max_depth=60)
clf.fit(X_train[:300], y_train[:300])

RandomForestClassifier(max_depth=60, n_estimators=20)

In [90]:
y_pred = clf.predict(X_test)
acc = accuracy(y_test, y_pred)
print(acc)
print(np.sum(y_pred == 0))

0.9365768896611643
724
